## Inbäddningar

I vårt tidigare exempel arbetade vi med högdimensionella bag-of-words-vektorer med längden `vocab_size`, och vi konverterade uttryckligen från lågdimensionella positionsrepresentationsvektorer till glesa one-hot-representationer. Denna one-hot-representation är inte minneseffektiv, dessutom behandlas varje ord oberoende av varandra, dvs. one-hot-kodade vektorer uttrycker ingen semantisk likhet mellan ord.

I denna enhet kommer vi att fortsätta utforska **News AG**-datasetet. För att börja, låt oss ladda data och hämta några definitioner från den tidigare notebooken.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## Vad är inbäddning?

Idén med **inbäddning** är att representera ord med lägre dimensionella täta vektorer, som på något sätt reflekterar ett ords semantiska betydelse. Vi kommer senare att diskutera hur man bygger meningsfulla ordinbäddningar, men för tillfället kan vi tänka på inbäddningar som ett sätt att minska dimensionen av en ordvektor.

Så, en inbäddningslager skulle ta ett ord som input och producera en utgångsvektor med en specificerad `embedding_size`. På ett sätt är det väldigt likt `Linear`-lagret, men istället för att ta en one-hot-kodad vektor, kan det ta ett ordnummer som input.

Genom att använda inbäddningslagret som det första lagret i vårt nätverk kan vi byta från bag-of-words till **embedding bag**-modellen, där vi först konverterar varje ord i vår text till motsvarande inbäddning och sedan beräknar någon aggregeringsfunktion över alla dessa inbäddningar, såsom `sum`, `average` eller `max`.

![Bild som visar en inbäddningsklassificerare för fem sekvensord.](../../../../../translated_images/sv/embedding-classifier-example.b77f021a7ee67eee.webp)

Vårt klassificerande neurala nätverk kommer att börja med ett inbäddningslager, sedan ett aggregeringslager och en linjär klassificerare ovanpå det:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### Hantering av variabel sekvensstorlek

Som ett resultat av denna arkitektur behöver minibatcher till vårt nätverk skapas på ett visst sätt. I den föregående enheten, när vi använde bag-of-words, hade alla BoW-tenstorer i en minibatch samma storlek `vocab_size`, oavsett den faktiska längden på vår textsekvens. När vi övergår till ordinbäddningar kommer vi att ha ett varierande antal ord i varje textprov, och när vi kombinerar dessa prover till minibatcher måste vi använda någon form av utfyllnad.

Detta kan göras genom att använda samma teknik som att tillhandahålla en `collate_fn`-funktion till datakällan:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### Träna inbäddningsklassificerare

Nu när vi har definierat en korrekt dataladdare kan vi träna modellen med hjälp av träningsfunktionen som vi definierade i föregående enhet:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **Observera**: Vi tränar endast på 25k poster här (mindre än en full epok) för att spara tid, men du kan fortsätta träna, skriva en funktion för att träna under flera epoker och experimentera med inlärningshastighetsparametern för att uppnå högre noggrannhet. Du bör kunna nå en noggrannhet på cirka 90 %.


### EmbeddingBag-lager och representation av sekvenser med variabel längd

I den tidigare arkitekturen behövde vi fylla ut alla sekvenser till samma längd för att passa in dem i en minibatch. Detta är inte det mest effektiva sättet att representera sekvenser med variabel längd - ett annat tillvägagångssätt skulle vara att använda en **offset**-vektor, som innehåller offset för alla sekvenser lagrade i en stor vektor.

![Bild som visar en offset-sekvensrepresentation](../../../../../translated_images/sv/offset-sequence-representation.eb73fcefb29b46ee.webp)

> **Note**: På bilden ovan visar vi en sekvens av tecken, men i vårt exempel arbetar vi med sekvenser av ord. Principen för att representera sekvenser med en offset-vektor förblir dock densamma.

För att arbeta med offset-representation använder vi [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html)-lagret. Det liknar `Embedding`, men det tar innehållsvektorn och offset-vektorn som indata, och det inkluderar också ett aggregeringslager, som kan vara `mean`, `sum` eller `max`.

Här är ett modifierat nätverk som använder `EmbeddingBag`:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

För att förbereda datasetet för träning behöver vi tillhandahålla en konverteringsfunktion som kommer att förbereda offsetvektorn:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

Observera att till skillnad från alla tidigare exempel accepterar vårt nätverk nu två parametrar: datavektor och offsetvektor, som har olika storlekar. På samma sätt ger vår dataladdare oss 3 värden istället för 2: både text- och offsetvektorer tillhandahålls som funktioner. Därför måste vi justera vår träningsfunktion något för att hantera detta:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## Semantiska inbäddningar: Word2Vec

I vårt tidigare exempel lärde sig modellens inbäddningslager att översätta ord till vektorrepresentationer, men denna representation hade inte mycket semantisk betydelse. Det skulle vara bra att lära sig en sådan vektorrepresentation där liknande ord eller synonymer motsvarar vektorer som ligger nära varandra baserat på någon vektordistans (t.ex. euklidisk distans).

För att göra detta behöver vi förträna vår inbäddningsmodell på en stor samling text på ett specifikt sätt. Ett av de första sätten att träna semantiska inbäddningar kallas [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Det bygger på två huvudsakliga arkitekturer som används för att skapa en distribuerad representation av ord:

 - **Continuous bag-of-words** (CBoW) — i denna arkitektur tränar vi modellen att förutsäga ett ord utifrån den omgivande kontexten. Givet ngrammet $(W_{-2},W_{-1},W_0,W_1,W_2)$ är målet för modellen att förutsäga $W_0$ utifrån $(W_{-2},W_{-1},W_1,W_2)$.
 - **Continuous skip-gram** är motsatsen till CBoW. Modellen använder det omgivande fönstret av kontextord för att förutsäga det aktuella ordet.

CBoW är snabbare, medan skip-gram är långsammare men gör ett bättre jobb med att representera sällsynta ord.

![Bild som visar både CBoW- och Skip-Gram-algoritmer för att konvertera ord till vektorer.](../../../../../translated_images/sv/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

För att experimentera med Word2Vec-inbäddningar förtränade på Google News-datasetet kan vi använda **gensim**-biblioteket. Nedan hittar vi de ord som är mest lik 'neural'.

> **Note:** När du först skapar ordvektorer kan nedladdningen ta lite tid!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Vi kan också beräkna vektorinbäddningar från ordet, för att användas vid träning av klassificeringsmodellen (vi visar endast de första 20 komponenterna av vektorn för tydlighet):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Det fantastiska med semantiska inbäddningar är att du kan manipulera vektorkodningen för att ändra semantiken. Till exempel kan vi be om att hitta ett ord vars vektorrepresentation är så nära som möjligt orden *kung* och *kvinna*, och så långt bort som möjligt från ordet *man*:


In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Både CBoW och Skip-Grams är "prediktiva" inbäddningar, eftersom de endast tar hänsyn till lokala kontexter. Word2Vec utnyttjar inte global kontext.

**FastText** bygger vidare på Word2Vec genom att lära sig vektorrepresentationer för varje ord och de tecken-n-gram som finns inom varje ord. Värdena för representationerna genomsnittas sedan till en vektor vid varje träningssteg. Även om detta lägger till mycket extra beräkning under förträningen, gör det att ordinbäddningar kan koda information på sub-ordsnivå.

En annan metod, **GloVe**, utnyttjar idén om samförekomstmatris och använder neurala metoder för att dekomponera samförekomstmatrisen till mer uttrycksfulla och icke-linjära ordvektorer.

Du kan experimentera med exemplet genom att ändra inbäddningar till FastText och GloVe, eftersom gensim stödjer flera olika modeller för ordinbäddning.


## Använda förtränade inbäddningar i PyTorch

Vi kan ändra exemplet ovan för att förfylla matrisen i vårt inbäddningslager med semantiska inbäddningar, såsom Word2Vec. Vi måste ta hänsyn till att vokabulärerna för förtränade inbäddningar och vår textkorpus sannolikt inte kommer att matcha, så vi kommer att initialisera vikterna för de saknade orden med slumpmässiga värden:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


Nu ska vi träna vår modell. Observera att tiden det tar att träna modellen är avsevärt längre än i det tidigare exemplet, på grund av den större storleken på inbäddningslagret och därmed ett mycket högre antal parametrar. Dessutom kan vi behöva träna vår modell på fler exempel om vi vill undvika överanpassning.


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

I vårt fall ser vi inte någon stor ökning i noggrannhet, vilket troligen beror på ganska olika ordförråd.  
För att lösa problemet med olika ordförråd kan vi använda en av följande lösningar:  
* Träna om word2vec-modellen på vårt ordförråd  
* Ladda vår dataset med ordförrådet från den förtränade word2vec-modellen. Ordförrådet som används för att ladda datasetet kan specificeras under laddningen.  

Den senare metoden verkar enklare, särskilt eftersom PyTorch `torchtext`-ramverket innehåller inbyggt stöd för inbäddningar. Vi kan till exempel skapa ett GloVe-baserat ordförråd på följande sätt:  


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


Laddat ordförråd har följande grundläggande operationer:
* `vocab.stoi` är en ordbok som låter oss konvertera ett ord till dess index i ordboken
* `vocab.itos` gör motsatsen - konverterar ett nummer till ett ord
* `vocab.vectors` är matrisen av inbäddningsvektorer, så för att få inbäddningen av ett ord `s` behöver vi använda `vocab.vectors[vocab.stoi[s]]`

Här är ett exempel på hur man manipulerar inbäddningar för att demonstrera ekvationen **snäll-man+kvinna = drottning** (jag var tvungen att justera koefficienten lite för att få det att fungera):


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

För att träna klassificeraren med hjälp av dessa inbäddningar måste vi först koda vår dataset med GloVe-ordförråd:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

Som vi har sett ovan, lagras alla vektorinbäddningar i matrisen `vocab.vectors`. Det gör det superenkelt att ladda dessa vikter i vikterna för inbäddningslagret genom enkel kopiering:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

Nu låt oss träna vår modell och se om vi får bättre resultat:


In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

En av anledningarna till att vi inte ser en betydande ökning i noggrannhet är att vissa ord från vår dataset saknas i den förtränade GloVe-ordlistan och därför i princip ignoreras. För att övervinna detta kan vi träna våra egna inbäddningar på vår dataset.


## Kontextuella Embeddingar

En viktig begränsning med traditionella förtränade embedding-representationer som Word2Vec är problemet med ords betydelseutredning. Även om förtränade embeddingar kan fånga en del av ordens betydelse i kontext, kodas alla möjliga betydelser av ett ord in i samma embedding. Detta kan orsaka problem i nedströmsmodeller, eftersom många ord, som ordet 'play', har olika betydelser beroende på sammanhanget de används i.

Till exempel har ordet 'play' i dessa två meningar ganska olika betydelser:
- Jag gick på en **pjäs** på teatern.
- John vill **leka** med sina vänner.

De förtränade embeddingarna ovan representerar båda dessa betydelser av ordet 'play' i samma embedding. För att övervinna denna begränsning behöver vi bygga embeddingar baserade på **språkmodellen**, som tränas på en stor textkorpus och *vet* hur ord kan sättas ihop i olika kontexter. Att diskutera kontextuella embeddingar ligger utanför ramen för denna handledning, men vi kommer tillbaka till dem när vi pratar om språkmodeller i nästa enhet.



---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, bör det noteras att automatiserade översättningar kan innehålla fel eller brister. Det ursprungliga dokumentet på dess originalspråk bör betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell human översättning. Vi ansvarar inte för eventuella missförstånd eller feltolkningar som kan uppstå vid användning av denna översättning.
